In [1]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, num_users,num_items, hidden_dim, layers):
        super(MLP,self).__init__()
        self.mlp_user_emb = nn.Embedding(num_users, hidden_dim)
        self.mlp_item_emb = nn.Embedding(num_items, hidden_dim)

        layers = [layers] if type(layers) is int else layers
        layers = [2*hidden_dim] + layers

        self.mlp_fc_layers = nn.ModuleList(nn.Linear(layers[i], layers[i+1]) for i in range(len(layers) -1))
        self.mlp_last = nn.Linear(layers[-1],1)

    def forward(self, user_indices, item_indices):
        u = self.mlp_user_emb(user_indices)
        i = self.mlp_item_emb(item_indices)
        output = torch.cat([u, i], dim = -1)
        for layer in self.mlp_fc_layers:
            output = layer(output)
            output = torch.relu(output)
        logit = self.mlp_last(output)
        result = torch.sigmoid(logit)
        return result    

In [2]:
import pandas as pd
from utils.data import SampleGenerator

ratings = pd.read_csv('dataset/ratings.csv')
ratings = ratings.rename(columns={'movieId': 'itemId'})

userId = list(set(ratings.userId))
new_userId = list(range(0,len(userId)))

df = pd.DataFrame({'userId':userId,'new_userId':new_userId})
ratings = pd.merge(ratings,df,how='left', on='userId')

itemId = list(set(ratings.itemId))
new_itemId = list(range(0,len(itemId)))

df = pd.DataFrame({'itemId':itemId,'new_itemId':new_itemId})
ratings = pd.merge(ratings,df,how='left', on='itemId')

ratings = ratings.drop(['userId', 'itemId'],axis = 1)
ratings = ratings.rename(columns={'new_userId':'userId', 'new_itemId':'itemId'})

data = SampleGenerator(ratings, implicit=True)
hidden_dim = 128
layers = [128]
lr = 0.001
batch_size = 2048
epochs = 15

num_users = data.num_users
num_items = data.num_items
num_negatives_train = 5
num_negatives_test = 500

cuda =  torch.cuda.is_available()

model = MLP(num_users, num_items, hidden_dim,layers)
criterion = nn.BCELoss()
optim = torch.optim.Adam(model.parameters(), lr)

if cuda:
    model.cuda()


C:\Users\hojin\Desktop\code\recsys-implementation-pytorch\1. Neural Collaborative Filtering\utils\data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ratings['rating'][ratings['rating'] >0] = 1.0


In [3]:
import os
from utils.eval import Evaluation

if not os.path.exists("./checkpoint"):
    os.mkdir("./checkpoint")

test_loader, negative_loader = data.instance_test_loader(num_negatives = num_negatives_test, batch_size = batch_size)
  
for epoch in range(1,epochs+1):

    train_loader = data.instance_a_train_loader(num_negatives=num_negatives_train, batch_size=batch_size)
    total_loss = 0
    for batch_id, batch in enumerate(train_loader):
        user, item, rating = batch[0], batch[1], batch[2]
        rating = rating.float()
        if cuda:
            user, item, rating = user.cuda(), item.cuda(), rating.cuda()
        optim.zero_grad()
        pred = model(user,item)
        loss = criterion(pred.view(-1), rating)
        loss.backward()
        optim.step()
        total_loss += loss.item()
    print("epoch{0} loss:{1:.4f}".format(epoch, total_loss))
  
    torch.save(model.state_dict(), "./checkpoint/mlp.pt")
  
    with torch.no_grad():
        test_users, test_items, test_preds = list(), list(), list()
        neg_users, neg_items, neg_preds = list(), list(), list()

        for batch in test_loader:
            user, item = batch[0], batch[1]
            test_users += user.data.view(-1).tolist()
            test_items += item.data.view(-1).tolist()

            if cuda:
                user, item = user.cuda(), item.cuda()
            pred = model(user,item)
            if cuda:
                pred = pred.cpu()

            test_preds += pred.data.view(-1).tolist()

        for batch in negative_loader:
            user, item = batch[0], batch[1]
            neg_users += user.data.view(-1).tolist()
            neg_items += item.data.view(-1).tolist()

            if cuda:
                user, item = user.cuda(), item.cuda()
            pred = model(user,item)
            if cuda:
                pred =  pred.cpu()

            neg_preds += pred.data.view(-1).tolist()

        eval = Evaluation([test_users, test_items, test_preds,
                           neg_users, neg_items, neg_preds])
        eval.print_eval_score_k(10)

epoch1 loss:90.9591
recall@10:0.4929, prec@10:0.1914
epoch2 loss:71.8468
recall@10:0.5714, prec@10:0.2411
epoch3 loss:64.4937
recall@10:0.5874, prec@10:0.2518
epoch4 loss:62.0526
recall@10:0.5930, prec@10:0.2525
epoch5 loss:60.6910
recall@10:0.5856, prec@10:0.2518
epoch6 loss:59.7658
recall@10:0.5908, prec@10:0.2528
epoch7 loss:58.9944
recall@10:0.5905, prec@10:0.2566
epoch8 loss:58.2342
recall@10:0.5925, prec@10:0.2561
epoch9 loss:57.6299
recall@10:0.5884, prec@10:0.2548
epoch10 loss:56.9360
recall@10:0.5881, prec@10:0.2560
epoch11 loss:56.1144
recall@10:0.5865, prec@10:0.2555
epoch12 loss:55.3230
recall@10:0.5859, prec@10:0.2544
epoch13 loss:54.3383
recall@10:0.5931, prec@10:0.2591
epoch14 loss:53.2932
recall@10:0.5878, prec@10:0.2591
epoch15 loss:51.9245
recall@10:0.5881, prec@10:0.2597
